In [6]:
rootF = 'D:/Data/Paper defocus/'

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
# from data import Dataset

from shared.save_load import load_obj
from shared.functions import fit_gaussian, get_ellipse_parameters, plot_scale_bar, load_image_as_on_MEA, corrcoef

# Load data

In [8]:
exp = 20211129

if exp == 20211208:
    trial = 67
    run_nb = 30
    n_cells = 29

elif exp == 20211129:
    trial = 20
    run_nb = 65
    n_cells = 21

elif exp == 20211207:
    trial = 37
    run_nb = 58
    n_cells = 32

In [9]:
results_folder = rootF + "/Modelling/exp" + str(exp) + '/cnn_{}_{}cells_tr{}/run_0{}/'.format(exp, n_cells, trial, run_nb) 

## Explained variance

#### Test set

In [10]:
test_observations = np.load(os.path.join(results_folder, 'test_observations.npy'))
test_predictions = np.load(os.path.join(results_folder, 'test_predictions.npy'))

explained_var_test_set = np.empty(n_cells)

test_y = test_observations
pred_y = np.mean(test_predictions, axis=0) # mean over the 30 repetitions
nb_conditions, nb_cells = test_y.shape[1:]

# Measure noise-corrected correlation 
even_test_y = np.mean(test_y[0::2, :, :], axis=0)  # mean over even repetitions
odd_test_y = np.mean(test_y[1::2, :, :], axis=0)  # mean over odd repetitions

for cell_nb in range(0, n_cells):
    reliability = corrcoef(even_test_y[:, cell_nb], odd_test_y[:, cell_nb])
    accuracy_odd = corrcoef(pred_y[:, cell_nb], odd_test_y[:, cell_nb])
    accuracy_even = corrcoef(pred_y[:, cell_nb], even_test_y[:, cell_nb])
    noise_corrected_correlation = 0.5*(accuracy_odd + accuracy_even)/np.sqrt(reliability)
    explained_var_test_set[cell_nb] = noise_corrected_correlation**2

In [11]:
test_predictions.shape

(30, 30, 21)

#### Blurred images

In [12]:
# Predictions
pred_data = np.load(rootF + "/Modelling/exp" + str(exp) + "/CNN_predictions_4_images_" + str(exp) + "_" + str(n_cells) \
         + "cells_tr" + str(trial) + "_run" + str(run_nb) + ".npz")

predictions = pred_data['predictions']
print ('predictions:', predictions.shape)

# Data
Imgs = ['1','2','3','5']
pupDiams = ['1.40','2.00']
sources = ['1','2','3','4'] 
sources_name = ['0','10','20','30']
dFs = ['+40','+30','+20','+10','+00','-10','-20','-30','-40'] 

Nimgs = len(Imgs)
NpupDiams = len(pupDiams)
Nsources = len(sources)
NdFs = len(dFs)

data_path = rootF + '/Modelling/exp' + str(exp) + '/'

blur_data = np.load(data_path + '{}_spike_counts_CNN_{}cells_tr{}_run{}.npz'.format(exp, n_cells, trial, run_nb))
spike_counts_mean = blur_data['spike_count_mean']
spike_counts_std = blur_data['spike_count_std']
spike_counts = blur_data['spike_count']
clusters = blur_data['clusters']

Ncells = len(clusters)

print (np.shape(spike_counts_mean))
print (clusters)

predictions: (4, 2, 9, 4, 21)
(21, 4, 2, 9, 4)
[  1  96 140 141 200 234 293 314 347 390 396 401 406 416 417 424 437 508
 558 583 614]


In [13]:
explained_var_blur_set = np.zeros(n_cells)

for iCell, cell in enumerate(clusters):
    test_y = spike_counts[iCell].reshape((288,25))[:,:-1]
    pred_y = predictions[:,:,:,:,iCell].reshape((288))
    nb_conditions = 25

    # Measure noise-corrected correlation 
    even_test_y = np.mean(test_y[:, 0::2], axis=1)  # mean over even repetitions
    odd_test_y = np.mean(test_y[:, 1::2], axis=1)  # mean over odd repetitions

    reliability = corrcoef(even_test_y, odd_test_y)
    accuracy_odd = corrcoef(pred_y, odd_test_y)
    accuracy_even = corrcoef(pred_y, even_test_y)
    noise_corrected_correlation = 0.5*(accuracy_odd + accuracy_even)/np.sqrt(reliability)
    explained_var_blur_set[iCell] = noise_corrected_correlation**2
    print (cell, noise_corrected_correlation**2)

1 0.9956432626765868
96 0.8846743833433172
140 0.9564957168433013
141 0.9822562709818055
200 0.9812789347258021
234 0.9632079605480304
293 0.9868583952268931
314 0.9532318358328034
347 0.9883541211943745
390 0.9974977180909501
396 0.9142506637428331
401 0.8280226143145039
406 0.9853026504609358
416 0.9756573120288461
417 0.9803955486483138
424 0.9964544855782862
437 0.9734644246616077
508 0.9917966471521908
558 0.355796046718513
583 0.9672665689785307
614 0.7335438643187043
